In [ ]:

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
df=session.table("NPS_SCORE.NPS_SCORE.NPS_SCORE")
df

In [ ]:
dataset=df.to_pandas()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.dropna(how='any',inplace=True)

In [ ]:
dataset.isnull().sum()

In [ ]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder()
dataset['PRODUCT_NAME']= label_encoder.fit_transform(dataset['PRODUCT_NAME']) 
dataset['PRODUCT_CATEGORY']= label_encoder.fit_transform(dataset['PRODUCT_CATEGORY']) 
dataset['PURCHASE_TYPE']= label_encoder.fit_transform(dataset['PURCHASE_TYPE']) 
dataset['SUPPORT_CHANNEL']=label_encoder.fit_transform(dataset['SUPPORT_CHANNEL'])
dataset['GENDER']=label_encoder.fit_transform(dataset['GENDER'])
dataset['ACQUISITION_CHANNEL']=label_encoder.fit_transform(dataset['ACQUISITION_CHANNEL'])


In [ ]:
#Text pre-processing
import re
import string
from textblob import TextBlob
strings=dataset['COMMENTS']
l=[]
def text_clean(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('[%s]'% re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text
for i in strings:
    s=text_clean(i)
    l.append(s)
dataset['COMMENTS']=l
dataset['COMMENTS']


Comment=[]
for i in dataset['COMMENTS']:
    blob = TextBlob(i)
    Comment.append(blob.sentiment.polarity)
dataset['COMMENTS']=Comment
dataset['COMMENTS']

In [ ]:
dataset['NUM_PURCHASES'].value_counts()
#l=list(map(lambda x : x if x > 0 else 8.447307608191855, dataset['NUM_PURCHASES']))
#dataset['NUM_PURCHASES']=l
p_25=dataset['NUM_PURCHASES'].quantile(0.25)
p_75=dataset['NUM_PURCHASES'].quantile(0.75)
iqr=p_75-p_25
upper_limit=1.5*iqr+p_75
lower_limit=p_25-1.5*iqr
print("UPPER LIMIT=",upper_limit)
print("LOWER_LIMIT",lower_limit)

#dataset['NUM_PURCHASES']=np.where(dataset['NUM_PURCHASES']>upper_limit,upper_limit,
#                                  np.where(dataset['NUM_PURCHASES']<lower_limit,lower_limit,
#                                          dataset['NUM_PURCHASES'])
#)
dataset = dataset[(dataset['NUM_PURCHASES'] >= lower_limit) & (dataset['NUM_PURCHASES'] <= upper_limit)]


In [ ]:
lower_INCOME=dataset['INCOME'].quantile(0.25)
upper_INCOME=dataset['INCOME'].quantile(0.75)
iqr1=upper_INCOME-lower_INCOME
lower_limit2=lower_INCOME-iqr1*1.5
upper_limit2=upper_INCOME+iqr1*1.5
#dataset['INCOME']=np.where(dataset['INCOME']>upper_INCOME,upper_INCOME,
#                            np.where(dataset['INCOME']<lower_INCOME,lower_INCOME,
#                                           dataset['INCOME']))
dataset = dataset[(dataset['INCOME'] >= lower_limit2) & (dataset['INCOME'] <= upper_limit2)]


In [ ]:
lower_IN=dataset['INTERACTION_FREQUENCY'].quantile(0.25)
upper_IN=dataset['INTERACTION_FREQUENCY'].quantile(0.75)
iqr1=upper_IN-lower_IN
lower_li=lower_IN-iqr1*1.5
upper_li=upper_IN+iqr1*1.5
#dataset['INTERACTION_FREQUENCY']=np.where(dataset['INTERACTION_FREQUENCY']>upper_li,upper_li,
#                            np.where(dataset['INTERACTION_FREQUENCY']<lower_li,lower_li,
#                                           dataset['INTERACTION_FREQUENCY'])
#                                 )
dataset = dataset[(dataset['INTERACTION_FREQUENCY'] >= lower_li) & (dataset['INTERACTION_FREQUENCY'] <= upper_li)]


In [ ]:
l=dataset['USAGE_FREQUENCY'].quantile(0.25)
u=dataset['USAGE_FREQUENCY'].quantile(0.75)
iqr1=u-l
low=l-iqr1*1.5
up=u+iqr1*1.5
#dataset['USAGE_FREQUENCY']=np.where(dataset['USAGE_FREQUENCY']>up,up,
#np.where(dataset['USAGE_FREQUENCY']<low,low,
#dataset['USAGE_FREQUENCY'])
#)
dataset = dataset[(dataset['USAGE_FREQUENCY'] >= low) & (dataset['USAGE_FREQUENCY'] <= up)]



In [ ]:
l=dataset['AVG_PURCHASE_VALUE'].quantile(0.25)
u=dataset['AVG_PURCHASE_VALUE'].quantile(0.75)
iqr1=u-l
low=l-iqr1*1.5
up=u+iqr1*1.5
#dataset['AVG_PURCHASE_VALUE']=np.where(dataset['AVG_PURCHASE_VALUE']>up,up,
#np.where(dataset['AVG_PURCHASE_VALUE']<low,low,
#dataset['AVG_PURCHASE_VALUE'])
#)
dataset = dataset[(dataset['AVG_PURCHASE_VALUE'] >= low) & (dataset['AVG_PURCHASE_VALUE'] <= up)]


In [ ]:
l=dataset['ONLINE_ACTIVITY'].quantile(0.25)
u=dataset['ONLINE_ACTIVITY'].quantile(0.75)
iqr1=u-l
low=l-iqr1*1.5
up=u+iqr1*1.5
#dataset['ONLINE_ACTIVITY']=np.where(dataset['ONLINE_ACTIVITY']>up,up,
#np.where(dataset['ONLINE_ACTIVITY']<low,low,
#dataset['ONLINE_ACTIVITY'])
#)
dataset = dataset[(dataset['ONLINE_ACTIVITY'] >= low) & (dataset['ONLINE_ACTIVITY'] <= up)]

In [ ]:
l1=dataset['CUSTOMER_SERVICE_INTERACTIONS'].quantile(0.25)
h1=dataset['CUSTOMER_SERVICE_INTERACTIONS'].quantile(0.75)
iqr=h1-l1
low1=l1-iqr*1.5
high1=h1+iqr*1.5
print(low1)
print(high1)
#dataset['CUSTOMER_SERVICE_INTERACTIONS']=np.where(dataset['CUSTOMER_SERVICE_INTERACTIONS']>h1,h1,
#np.where(dataset['CUSTOMER_SERVICE_INTERACTIONS']<l1,l1,
#dataset['CUSTOMER_SERVICE_INTERACTIONS']))
dataset = dataset[(dataset['CUSTOMER_SERVICE_INTERACTIONS'] >= low1) & (dataset['CUSTOMER_SERVICE_INTERACTIONS'] <= high1)]

In [ ]:
lower_25=dataset['DISCOUNTS_RECEIVED'].quantile(0.25)
upper_75=dataset['DISCOUNTS_RECEIVED'].quantile(0.75)
iqr=upper_75-lower_25
lower_limit1=lower_25-iqr*1.5
upper_limit1=upper_75+iqr*1.5
#dataset['DISCOUNTS_RECEIVED']=np.where(dataset['DISCOUNTS_RECEIVED']>upper_limit1,upper_limit1,
#                           np.where(dataset['DISCOUNTS_RECEIVED']<lower_limit1,lower_limit1,
#                                           dataset['DISCOUNTS_RECEIVED'])
#                                )
dataset = dataset[(dataset['DISCOUNTS_RECEIVED'] >= lower_limit1) & (dataset['DISCOUNTS_RECEIVED'] <= upper_limit1)]

In [ ]:
dataset.drop('RESPONDENT',inplace=True,axis=1)
dataset.drop('LOCATION',inplace=True,axis=1)

In [ ]:
current_date=pd.Timestamp.now()
dataset['REGISTRATION_DATE']=pd.to_datetime(dataset['REGISTRATION_DATE'])
dataset['LAST_PURCHASE_DATE']=pd.to_datetime(dataset['LAST_PURCHASE_DATE'])
dataset['TENURE']=(current_date-dataset['REGISTRATION_DATE']).dt.days
dataset['RECENT']=(current_date-dataset['LAST_PURCHASE_DATE']).dt.days


In [ ]:
dataset.drop(['REGISTRATION_DATE', 'LAST_PURCHASE_DATE'], axis=1, inplace=True)

In [ ]:
dataset=dataset[(dataset['AGE'] >= 21) & (dataset['AGE'] <= 60)]

In [ ]:
dataset['NPS_CATEGORY'].replace(to_replace=['Passive','Detractor','Promoter'],value=[0,1,2],inplace=True)

In [ ]:
lower_25=dataset['RECENT'].quantile(0.25)
upper_75=dataset['RECENT'].quantile(0.75)
iqr=upper_75-lower_25
lower_limit1=lower_25-iqr*1.5
upper_limit1=upper_75+iqr*1.5
#dataset['DISCOUNTS_RECEIVED']=np.where(dataset['DISCOUNTS_RECEIVED']>upper_limit1,upper_limit1,
#                           np.where(dataset['DISCOUNTS_RECEIVED']<lower_limit1,lower_limit1,
#                                           dataset['DISCOUNTS_RECEIVED'])
#                                )
dataset = dataset[(dataset['RECENT'] >= lower_limit1) & (dataset['RECENT'] <= upper_limit1)]

In [ ]:
lower_25=dataset['TENURE'].quantile(0.25)
upper_75=dataset['TENURE'].quantile(0.75)
iqr=upper_75-lower_25
lower_limit1=lower_25-iqr*1.5
upper_limit1=upper_75+iqr*1.5
#dataset['DISCOUNTS_RECEIVED']=np.where(dataset['DISCOUNTS_RECEIVED']>upper_limit1,upper_limit1,
#                           np.where(dataset['DISCOUNTS_RECEIVED']<lower_limit1,lower_limit1,
#                                           dataset['DISCOUNTS_RECEIVED'])
#                                )
dataset = dataset[(dataset['TENURE'] >= lower_limit1) & (dataset['TENURE'] <= upper_limit1)]

In [ ]:
dataset.drop('COMMENTS',axis=1,inplace=True)
dataset['NPS_CATEGORY']=dataset['NPS_CATEGORY'].astype(int)
mask = dataset.lt(0).any(axis=1)

# Filter out rows with any negative value
dataset= dataset[~mask]

In [ ]:
dataset.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
y=dataset['NPS_CATEGORY']
dataset.drop('NPS_CATEGORY',inplace=True,axis=1)
scaler=StandardScaler()
x=dataset
scaled_x=scaler.fit_transform(x)
x_scale=pd.DataFrame(scaled_x,columns=x.columns)
x_scale.head()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
m=ExtraTreesClassifier()
ordered_rank=m.fit(x_scale,y)
rank=pd.Series(m.feature_importances_,index=x_scale.columns)
rank.nlargest(10).plot(kind="bar")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X=x_scale[['RESPONSE_TIME','INCOME','NUM_PURCHASES','AGE','AVG_PURCHASE_VALUE','RECENT']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import xgboost
params={
    "learning_rate":[0.05,0.10,0.15,0.20,0.25,0.30],
    "max_depth":[3,4,5,6,8,10,12,15],
    "min_child_weight":[1,3,5,7],
    "gamma":[130,50,40,100],
    "colsample_bytree":[0.3,0.4,0.5,0.6,0.7]
}
classifier=xgboost.XGBClassifier()
random_search1=RandomizedSearchCV(classifier,param_distributions=params,n_iter=50,scoring="accuracy",n_jobs=1,cv=5,verbose=2)
random_search1.fit(X_train, y_train)


In [ ]:
random_search1.best_params_

In [ ]:
cf=xgboost.XGBClassifier(min_child_weight=7,max_depth=4,learning_rate=0.15,eta=0.1,gamma=100,colsample_bytree=0.5)

In [ ]:
best_model = random_search1.best_estimator_
accuracy = best_model.score(X_test, y_test)
print("Accuracy on test set:", accuracy)